In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import logging
import os
from util import getDefaultHyperparams
import time
from DayDataloaders import create_Dataloaders
# Import the metrics file
from SequenceLoss import SequenceLoss
from network import Net
import pickle



In [5]:
params = pickle.load(open('trainOutputs/hyperparam_525268.pickle', 'rb'))
for key in params:
    print(key, params[key]) 

batch_size 50
epochs 200
batchesPerVal 10
updateVal 20
rnnBinSize 2
skipLen 5
smoothInputs 1
cvPart HeldOutBlocks
outputDelay 50
directionality unidirectional
loss crossentropy
learning_rate 0.001
scheduler LambdaLR
scheduler_gamma 0.9
scheduler_step_size 10
dropout 0
weight_decay 0.0
noisy_threshold 0.2
var_dropout_s 0
var_dropout_t 0
zoneout False
constantOffsetSD 0
randomWalkSD 0
whiteNoiseSD 0.6
train_val_timeSteps 1200
test_timeSteps 7500
batchnorm none
neuron_count 512
use_bias True
steps 10
warmup_steps 2
network_type default
reset_by_subtraction True
surrogate_gradient square
init_u zero
Vth 0.4
Vth_init_range 0.1
Vth_trainable False
tau 0.6
tau_init_range 0.1
tau_init_distribution uniform
tau_trainable True
tau_sigmoid False
constrain_method forward
output_type vel
static_quantisation False
quantize_weight False
quantize_bias False
quantize_accumulation False
quantize_activation False
quantize_membrane_pot False
quantize_input False
quantize_input_scaling False
quantisation_sc

In [1]:
import torch
import torch.nn as nn
import numpy as np


# Create the Conv1d layer
conv = nn.Conv1d(1, 1, kernel_size=5, padding=5//2)

# Create a sample input tensor
input_array = np.arange(5120).reshape(10, 512)
input_tensor = torch.tensor(input_array, dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(1)  # Add a dummy sequence_length dimension
print("Input tensor shape:", input_tensor.shape)

# Apply the convolution operation
output_tensor = conv(input_tensor)
output_tensor = output_tensor.squeeze(1)  # Remove the dummy sequence_length dimension

# Print the shapes of the input and output tensors
print("Output tensor shape:", output_tensor.shape)

print("Output tensor:", output_tensor)

Input tensor shape: torch.Size([10, 1, 512])
Output tensor shape: torch.Size([10, 512])
Output tensor: tensor([[ 3.5886e-01,  5.5119e-01,  4.5247e-01,  ...,  9.8427e+01,
         -1.4705e+01, -4.4690e+01],
        [ 9.8831e+01, -4.9991e+01,  9.9393e+01,  ...,  1.9737e+02,
         -2.9090e+01, -8.9031e+01],
        [ 1.9730e+02, -1.0053e+02,  1.9833e+02,  ...,  2.9631e+02,
         -4.3475e+01, -1.3337e+02],
        ...,
        [ 6.8966e+02, -3.5324e+02,  6.9304e+02,  ...,  7.9101e+02,
         -1.1540e+02, -3.5508e+02],
        [ 7.8814e+02, -4.0379e+02,  7.9198e+02,  ...,  8.8995e+02,
         -1.2978e+02, -3.9942e+02],
        [ 8.8661e+02, -4.5433e+02,  8.9092e+02,  ...,  9.8889e+02,
         -1.4417e+02, -4.4376e+02]], grad_fn=<SqueezeBackward1>)


In [24]:
hyperparams = getDefaultHyperparams()

if hyperparams['system'] == 'Linux':
    prepared_data_dir = '/scratch/sem24f8/dataset/'
else:
    prepared_data_dir = 'dataset/'

hyperparams['prepared_data_dir'] = prepared_data_dir

hyperparams['n_channels'] = 192
hyperparams['n_outputs'] = 32
hyperparams['batchSize'] = 2

# Load the prepared data
print('Loading prepared data from dir')
prepared_data = torch.load(prepared_data_dir + 'prepared_data.pth')
print('Data loaded')

manual_prep = False
# loading the testing dataset and creating a DataLoader for each day
Test_finite_loader = create_Dataloaders(manual_prep, hyperparams, days=np.arange(10), mode='testing')
test_loaders = Test_finite_loader.getDataloaders()
viable_test_days = Test_finite_loader.getViableDays()
print('Testing dataloaders ready')



Loading prepared data from dir


KeyboardInterrupt: 

In [ ]:
# Load the model
id = '107776'

from util import testModel

device = torch.device('cpu')
if torch.cuda.is_available():
    print('GPU available')
    device = torch.device(hyperparams['device'])
print(f'Device: {device}')

model = Net(hyperparams).to(device)
state_dict = torch.load(f'Model/model_inf_{id}.pth', map_location=device)
model.load_state_dict(state_dict)


# test the model
criterion = SequenceLoss(hyperparams)
testModel(model, test_loaders, viable_test_days, criterion, hyperparams, device)

GPU available
Device: cuda:0
Testing on day t5.2019.11.25


TypeError: only integer scalar arrays can be converted to a scalar index